In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
from itertools import combinations, permutations
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
race_results = pd.read_pickle("../data/original_data/race_results.pickle")

In [4]:
def preprocessing(race_results):
    df = race_results.copy()
    # 着順に数字以外の文字列が含まれているものを取り除く
    df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
    df.dropna(subset=['着順'], inplace=True)
    df['着順'] = df['着順'].astype(int)
    df['rank'] = df['着順'].map(lambda x:1 if x<4 else 0)

    # 性齢を性と年齢に分ける
    df["gender"] = df["性齢"].map(lambda x: str(x)[0])
    df["age"] = df["性齢"].map(lambda x: str(x)[1:]).astype(int)

    # 馬体重を体重と体重変化に分ける
    df["weight"] = df["馬体重"].str.split("(", expand=True)[0]
    df["change_weight"] = df["馬体重"].str.split("(", expand=True)[1].str[:-1]
    df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
    df['change_weight'] = pd.to_numeric(df['change_weight'], errors='coerce')
    df.dropna(subset=['weight',"change_weight"], inplace=True)
    df["weight"] = df["weight"].astype(int)
    df["change_weight"] = df["change_weight"].astype(int)
    
    #出走数追加
    df['n_horses'] = df.index.map(df.index.value_counts())

    # 単勝をfloatに変換
    df["単勝"] = df["単勝"].astype(float)

    # 距離は10の位を切り捨てる
    df["course_len"] = df["course_len"].astype(float) // 100

    # 人気をfloatに変換
    df["人気"] = df["人気"].astype(float)

    # 日付をdatetimeに変更
    df["date"] = pd.to_datetime(df["date"], format="%Y年%m月%d日")

    # 枠番をintに変更
    df["枠番"] = df["枠番"].astype(int)

    # 不要な列を削除
    df.drop(["タイム", "着差", "調教師", "性齢", "馬体重", '馬名', '騎手'], axis=1, inplace=True)
    #開催場所
    df['place'] = df.index.map(lambda x:str(x)[4:6])
	
    return df

In [5]:
p_race_results = preprocessing(race_results)

In [6]:
horse_results = pd.read_pickle("../data/original_data/horse_results.pickle")

In [7]:
def horse_preprocessing(horse_results):
  df = horse_results[['日付', '着順', '賞金', "頭数"]]

  # 数値以外の欠損値を削除
  df["着順"] = pd.to_numeric(df["着順"], errors="coerce")
  df.dropna(subset=["着順"], inplace=True)
  df["着順"] = df["着順"].astype(int)

  # 賞金のNANを0で埋める
  df["賞金"].fillna(0, inplace=True)

  # (1 - 着順/頭数)の計算
  df["着順/頭数"] = (1 - df["着順"] / df["頭数"]) * 100

  # 日付型に変換
  df["date"] = pd.to_datetime(df["日付"])
  df.drop(["日付"], axis=1, inplace=True)
  return df

In [8]:
p_horse_results = horse_preprocessing(horse_results)

C:\Users\81902\AppData\Local\Temp\ipykernel_11752\1359851031.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["着順"] = pd.to_numeric(df["着順"], errors="coerce")
C:\Users\81902\AppData\Local\Temp\ipykernel_11752\1359851031.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["着順"], inplace=True)
C:\Users\81902\AppData\Local\Temp\ipykernel_11752\1359851031.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [9]:
def Merge_race_with_horse(race_results, horse_results, n_samples={3, 5, 7, "all"}):
  df_r = race_results.copy()
  df_h = horse_results.copy()
  date_list = df_r["date"].unique()
  for n_sample in n_samples:
    merged_all_df = pd.DataFrame()
    for date in date_list:
      df = df_r[df_r["date"] == date]
      horse_id_list = df["horse_id"]
      target_df = df_h.loc[horse_id_list]
      if n_sample == "all":
        filtered_df = target_df[target_df["date"] < date]
      else:
        filtered_df = target_df[target_df["date"] < date].sort_values("date", ascending = False).groupby(level=0).head(n_sample)
      average_df = filtered_df.groupby(level=0)[["着順", "賞金", "着順/頭数"]].mean()
      average_df.rename(columns={"着順":"着順_{}R".format(n_sample), "賞金":"賞金_{}R".format(n_sample) ,"着順/頭数":"着順/頭数_{}R".format(n_sample)},inplace=True)
      merged_df = pd.merge(df, average_df, how="left", left_on="horse_id", right_index=True)
      merged_all_df = pd.concat([merged_all_df, merged_df])
    df_r = merged_all_df.copy()
  return df_r

In [10]:
m_race_results = Merge_race_with_horse(p_race_results, p_horse_results)

In [20]:
m_horse_id_list = m_race_results[m_race_results["着順_allR"].isnull()]["horse_id"].unique()

In [21]:
horse_id_list = p_race_results["horse_id"].unique()

In [24]:
len(horse_id_list)

26190

In [25]:
len(m_horse_id_list)

19139

In [31]:
m_horse_id_list

array(['2016104246', '2017105100', '2017104177', ..., '2020104795',
       '2020100913', '2020103115'], dtype=object)

In [27]:
len(set(horse_id_list) & set(m_horse_id_list))

19139

In [34]:
p_horse_results[p_horse_results.index == "2017104177"]

,着順,賞金,頭数,着順/頭数,date
2017104177,12,0.0,14.0,14.285714,2021-03-03
2017104177,15,0.0,16.0,6.250000,2020-12-19
2017104177,14,0.0,15.0,6.666667,2020-07-11
2017104177,1,510.0,16.0,93.750000,2020-06-21
2017104177,6,0.0,16.0,62.500000,2020-06-06
2017104177,2,200.0,15.0,86.666667,2020-05-09
2017104177,2,200.0,16.0,87.500000,2020-04-12
2017104177,10,0.0,18.0,44.444444,2020-03-08
2017104177,8,0.0,14.0,42.857143,2019-10-26
2017104177,10,0.0,15.0,33.333333,2019-09-15


In [35]:
p_race_results[p_race_results["horse_id"] == "2017104177"]

,着順,枠番,馬番,斤量,単勝,人気,course_len,weather,race_type,ground_state,date,horse_id,jockey_id,rank,gender,age,weight,change_weight,n_horses,place
201901010105,2,7,8,54.0,67.0,8.0,15.0,晴,芝,良,2019-07-27,2017104177,01109,1,牡,2,448,0,9,01
201901020209,4,6,6,54.0,6.2,2.0,15.0,晴,芝,良,2019-08-18,2017104177,01109,0,牡,2,456,8,8,01
201901020610,12,6,9,54.0,51.2,8.0,12.0,晴,芝,良,2019-09-01,2017104177,01109,0,牡,2,458,2,14,01
201904030505,8,8,14,52.0,7.7,4.0,14.0,曇,芝,重,2019-10-26,2017104177,01164,0,牡,2,464,2,13,04
201906040403,10,6,11,54.0,24.9,6.0,18.0,晴,芝,良,2019-09-15,2017104177,01109,0,牡,2,462,4,15,06
202003020312,14,7,13,54.0,25.2,10.0,17.0,小雨,ダート,稍重,2020-07-11,2017104177,01143,0,牡,3,480,0,15,03
202005020502,2,5,8,56.0,4.9,3.0,13.0,晴,ダート,良,2020-05-09,2017104177,01092,1,牡,3,476,-4,15,05
202005030103,6,4,7,56.0,6.2,4.0,13.0,曇,ダート,良,2020-06-06,2017104177,01092,0,牡,3,476,0,16,05
202005030604,1,8,16,56.0,8.6,4.0,16.0,晴,ダート,稍重,2020-06-21,2017104177,01092,1,牡,3,480,4,16,05
202006030604,2,7,13,56.0,7.1,4.0,12.0,曇,ダート,稍重,2020-04-12,2017104177,01092,1,牡,3,480,4,16,06


In [232]:
d_df = pd.get_dummies(m_race_results)

In [28]:
d_df

NameError: name 'd_df' is not defined

In [234]:
#時系列に沿って訓練データとテストデータに分ける関数
def split_data(df, test_size=0.3):
    sorted_id_list = df.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df.loc[train_id_list]
    test = df.loc[test_id_list]
    return train, test

In [235]:
train, valid  = split_data(d_df)

In [236]:
train.shape

(135147, 26511)

In [304]:
valid

,着順,枠番,馬番,斤量,単勝,人気,course_len,date,rank,age,weight,change_weight,n_horses,着順_allR,賞金_allR,着順/頭数_allR,着順_3R,賞金_3R,着順/頭数_3R,着順_5R,賞金_5R,着順/頭数_5R,着順_7R,賞金_7R,着順/頭数_7R,...,jockey_id_05622,jockey_id_05623,jockey_id_05624,jockey_id_05625,jockey_id_05626,jockey_id_05627,jockey_id_05628,jockey_id_05629,jockey_id_05630,jockey_id_05631,jockey_id_05632,jockey_id_05633,gender_セ,gender_牝,gender_牡,place_01,place_02,place_03,place_04,place_05,place_06,place_07,place_08,place_09,place_10
202109050103,1,2,4,55.0,5.7,3.0,16.0,2021-11-06,1,2,472,-4,17,7.500000,90.000000,47.500000,7.500000,90.000000,47.500000,7.50,90.0,47.500000,7.500000,90.000000,47.500000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,2,6,11,55.0,18.7,5.0,16.0,2021-11-06,1,2,522,2,17,5.000000,70.000000,68.750000,5.000000,70.000000,68.750000,5.00,70.0,68.750000,5.000000,70.000000,68.750000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,3,1,2,55.0,12.9,4.0,16.0,2021-11-06,1,2,452,4,17,4.000000,110.000000,75.000000,4.000000,110.000000,75.000000,4.00,110.0,75.000000,4.000000,110.000000,75.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,4,7,13,54.0,2.0,1.0,16.0,2021-11-06,0,2,496,-8,17,2.250000,202.500000,76.515152,2.333333,176.666667,74.242424,2.25,202.5,76.515152,2.250000,202.500000,76.515152,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
202109050103,5,6,12,55.0,448.1,16.0,16.0,2021-11-06,0,2,442,2,17,10.000000,0.000000,33.333333,10.000000,0.000000,33.333333,10.00,0.0,33.333333,10.000000,0.000000,33.333333,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202306010812,5,6,6,57.0,8.2,5.0,16.0,2023-01-22,0,4,512,-4,9,6.000000,226.714286,53.350221,10.000000,0.000000,23.232323,7.00,266.0,46.356976,6.000000,226.714286,53.350221,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,6,7,7,56.0,4.3,2.0,16.0,2023-01-22,0,5,476,12,9,5.083333,241.366667,64.518782,6.666667,202.000000,53.212670,5.40,275.2,57.344268,4.571429,285.200000,64.812810,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
202306010812,7,5,5,58.0,74.2,9.0,16.0,2023-01-22,0,5,478,2,9,9.105263,110.105263,29.573774,9.000000,0.000000,36.698718,9.40,0.0,26.463675,10.571429,0.000000,21.680403,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,8,8,8,56.0,47.3,8.0,16.0,2023-01-22,0,5,476,2,9,4.571429,206.700000,71.237245,8.666667,37.000000,42.592593,7.60,78.2,50.019841,6.000000,219.542857,61.125283,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [238]:
X_train = train.drop(['rank', 'date', "着順"], axis=1)
y_train = train['rank']
X_valid = valid.drop(['rank', 'date', "着順"], axis=1)
y_valid = valid['rank']

In [239]:
import optuna.integration.lightgbm as lgb_o

In [165]:
# #データセットを作成
# lgb_train = lgb_o.Dataset(X_train.values, y_train.values)
# lgb_valid = lgb_o.Dataset(X_valid.values, y_valid.values)

# params = {
#     'objective': 'binary', #今回は0or1の二値予測なのでbinaryを指定
#     'random_state': 100
# }

# #チューニング実行
# lgb_clf_o = lgb_o.train(params, lgb_train,
#                         valid_sets=(lgb_train, lgb_valid),
#                         verbose_eval=100,
#                         early_stopping_rounds=10,
#                         optuna_seed=100 #optunaのseed固定
#                         )

[I 2023-01-30 02:30:58,014] A new study created in memory with name: no-name-2b89d31f-afb5-4512-b9d3-e7af68de781f




c:\keiba\keiba\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\keiba\keiba\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1662, number of negative: 6071
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001382 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11638
[LightGBM] [Info] Number of data points in the train set: 7733, number of used features: 5576






[I 2023-01-30 02:30:59,681] Trial 0 finished with value: 0.40972776524762594 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 0 with value: 0.40972776524762594.




[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.214923 -> initscore=-1.295502
[LightGBM] [Info] Start training from score -1.295502
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[29]	valid_0's binary_logloss: 0.355943	valid_1's binary_logloss: 0.409728
[LightGBM] [Info] Number of positive: 1662, number of negative: 6071
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11638
[LightGBM] [Info] Number of data points in the train set: 7733, number of used features: 5576






[I 2023-01-30 02:31:01,348] Trial 1 finished with value: 0.41360824790740874 and parameters: {'feature_fraction': 0.5}. Best is trial 0 with value: 0.40972776524762594.




...

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.214923 -> initscore=-1.295502
[LightGBM] [Info] Start training from score -1.295502
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[66]	valid_0's binary_logloss: 0.398489	valid_1's binary_logloss: 0.402757
[LightGBM] [Info] Number of positive: 1662, number of negative: 6071
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001067 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11638
[LightGBM] [Info] Number of data points in the train set: 7733, number of used features: 5576






[I 2023-01-30 02:32:45,741] Trial 63 finished with value: 0.40269799389611555 and parameters: {'min_child_samples': 5}. Best is trial 61 with value: 0.40269799389611555.




[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.214923 -> initscore=-1.295502
[LightGBM] [Info] Start training from score -1.295502
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[88]	valid_0's binary_logloss: 0.396845	valid_1's binary_logloss: 0.402698
[LightGBM] [Info] Number of positive: 1662, number of negative: 6071
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001609 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11638
[LightGBM] [Info] Number of data points in the train set: 7733, number of used features: 5576






[I 2023-01-30 02:32:47,407] Trial 64 finished with value: 0.40269799389611555 and parameters: {'min_child_samples': 10}. Best is trial 61 with value: 0.40269799389611555.
min_data_in_leaf, val_score: 0.402698: 100%|##########| 5/5 [00:08<00:00,  1.73s/it]

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.214923 -> initscore=-1.295502
[LightGBM] [Info] Start training from score -1.295502
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[88]	valid_0's binary_logloss: 0.396845	valid_1's binary_logloss: 0.402698


In [240]:
params = {'objective': 'binary',
 'random_state': 100,
 'feature_pre_filter': False,
 'lambda_l1': 9.490245203532942e-07,
 'lambda_l2': 6.421168438428032,
 'num_leaves': 3,
 'feature_fraction': 0.9520000000000001,
 'bagging_fraction': 0.7260429650751228,
 'bagging_freq': 2,
 'min_child_samples': 20,
 'num_iterations': 1000,}

In [241]:
lgb_clf = lgb.LGBMClassifier(**params)
lgb_clf.fit(X_train.values, y_train.values)

c:\keiba\keiba\env\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9520000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9520000000000001
[LightGBM] [Warning] bagging_fraction is set=0.7260429650751228, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7260429650751228
[LightGBM] [Warning] lambda_l1 is set=9.490245203532942e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.490245203532942e-07
[LightGBM] [Warning] lambda_l2 is set=6.421168438428032, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.421168438428032
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


LGBMClassifier(bagging_fraction=0.7260429650751228, bagging_freq=2,
               feature_fraction=0.9520000000000001, feature_pre_filter=False,
               lambda_l1=9.490245203532942e-07, lambda_l2=6.421168438428032,
               num_iterations=1000, num_leaves=3, objective='binary',
               random_state=100)

In [245]:
# モデルの保存
import pickle
file = 'trained_model.pkl'
pickle.dump(lgb_clf, open(file, 'wb'))

In [246]:
y_pred_train = lgb_clf.predict_proba(X_train)[:,1]

In [247]:
y_pred_train

array([0.8032649 , 0.18645396, 0.21127822, ..., 0.13426204, 0.06753806,
       0.05473325])

In [248]:
y_pred = lgb_clf.predict_proba(X_valid)[:,1]

In [302]:
X_train

,枠番,馬番,斤量,単勝,人気,course_len,age,weight,change_weight,n_horses,着順_allR,賞金_allR,着順/頭数_allR,着順_3R,賞金_3R,着順/頭数_3R,着順_5R,賞金_5R,着順/頭数_5R,着順_7R,賞金_7R,着順/頭数_7R,weather_小雨,weather_小雪,weather_晴,...,jockey_id_05622,jockey_id_05623,jockey_id_05624,jockey_id_05625,jockey_id_05626,jockey_id_05627,jockey_id_05628,jockey_id_05629,jockey_id_05630,jockey_id_05631,jockey_id_05632,jockey_id_05633,gender_セ,gender_牝,gender_牡,place_01,place_02,place_03,place_04,place_05,place_06,place_07,place_08,place_09,place_10
201908010105,7,14,56.0,1.5,1.0,18.0,3,490,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
201908010105,2,4,54.0,18.9,5.0,18.0,3,474,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
201908010105,1,2,53.0,18.1,4.0,18.0,3,464,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
201908010105,4,7,56.0,5.5,2.0,18.0,3,456,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
201908010105,8,16,54.0,115.6,12.0,18.0,3,398,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202109050102,6,7,55.0,3.4,1.0,12.0,2,498,8,11,3.0,180.0,62.500000,3.0,180.0,62.500000,3.0,180.0,62.500000,3.0,180.0,62.500000,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050102,1,1,55.0,308.6,11.0,12.0,2,476,4,11,9.4,0.0,11.914530,9.0,0.0,16.153846,9.4,0.0,11.914530,9.4,0.0,11.914530,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050102,3,3,54.0,33.6,7.0,12.0,2,446,4,11,11.5,0.0,8.823529,11.5,0.0,8.823529,11.5,0.0,8.823529,11.5,0.0,8.823529,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
202109050102,8,10,55.0,56.6,8.0,12.0,2,468,0,11,15.0,0.0,5.882353,15.0,0.0,5.882353,15.0,0.0,5.882353,15.0,0.0,5.882353,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [301]:
X_valid

,枠番,馬番,斤量,単勝,人気,course_len,age,weight,change_weight,n_horses,着順_allR,賞金_allR,着順/頭数_allR,着順_3R,賞金_3R,着順/頭数_3R,着順_5R,賞金_5R,着順/頭数_5R,着順_7R,賞金_7R,着順/頭数_7R,weather_小雨,weather_小雪,weather_晴,...,jockey_id_05622,jockey_id_05623,jockey_id_05624,jockey_id_05625,jockey_id_05626,jockey_id_05627,jockey_id_05628,jockey_id_05629,jockey_id_05630,jockey_id_05631,jockey_id_05632,jockey_id_05633,gender_セ,gender_牝,gender_牡,place_01,place_02,place_03,place_04,place_05,place_06,place_07,place_08,place_09,place_10
202109050103,2,4,55.0,5.7,3.0,16.0,2,472,-4,17,7.500000,90.000000,47.500000,7.500000,90.000000,47.500000,7.50,90.0,47.500000,7.500000,90.000000,47.500000,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,6,11,55.0,18.7,5.0,16.0,2,522,2,17,5.000000,70.000000,68.750000,5.000000,70.000000,68.750000,5.00,70.0,68.750000,5.000000,70.000000,68.750000,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,1,2,55.0,12.9,4.0,16.0,2,452,4,17,4.000000,110.000000,75.000000,4.000000,110.000000,75.000000,4.00,110.0,75.000000,4.000000,110.000000,75.000000,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,7,13,54.0,2.0,1.0,16.0,2,496,-8,17,2.250000,202.500000,76.515152,2.333333,176.666667,74.242424,2.25,202.5,76.515152,2.250000,202.500000,76.515152,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
202109050103,6,12,55.0,448.1,16.0,16.0,2,442,2,17,10.000000,0.000000,33.333333,10.000000,0.000000,33.333333,10.00,0.0,33.333333,10.000000,0.000000,33.333333,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202306010812,6,6,57.0,8.2,5.0,16.0,4,512,-4,9,6.000000,226.714286,53.350221,10.000000,0.000000,23.232323,7.00,266.0,46.356976,6.000000,226.714286,53.350221,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,7,7,56.0,4.3,2.0,16.0,5,476,12,9,5.083333,241.366667,64.518782,6.666667,202.000000,53.212670,5.40,275.2,57.344268,4.571429,285.200000,64.812810,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
202306010812,5,5,58.0,74.2,9.0,16.0,5,478,2,9,9.105263,110.105263,29.573774,9.000000,0.000000,36.698718,9.40,0.0,26.463675,10.571429,0.000000,21.680403,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,8,8,56.0,47.3,8.0,16.0,5,476,2,9,4.571429,206.700000,71.237245,8.666667,37.000000,42.592593,7.60,78.2,50.019841,6.000000,219.542857,61.125283,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [250]:
result_data = lgb_clf.predict_proba(X_valid)[:, 1]

In [303]:
result_data

array([0.43207796, 0.20913978, 0.28305063, ..., 0.06539425, 0.10492492,
       0.09392313])

In [251]:
X_valid

,枠番,馬番,斤量,単勝,人気,course_len,age,weight,change_weight,n_horses,着順_allR,賞金_allR,着順/頭数_allR,着順_3R,賞金_3R,着順/頭数_3R,着順_5R,賞金_5R,着順/頭数_5R,着順_7R,賞金_7R,着順/頭数_7R,weather_小雨,weather_小雪,weather_晴,...,jockey_id_05622,jockey_id_05623,jockey_id_05624,jockey_id_05625,jockey_id_05626,jockey_id_05627,jockey_id_05628,jockey_id_05629,jockey_id_05630,jockey_id_05631,jockey_id_05632,jockey_id_05633,gender_セ,gender_牝,gender_牡,place_01,place_02,place_03,place_04,place_05,place_06,place_07,place_08,place_09,place_10
202109050103,2,4,55.0,5.7,3.0,16.0,2,472,-4,17,7.500000,90.000000,47.500000,7.500000,90.000000,47.500000,7.50,90.0,47.500000,7.500000,90.000000,47.500000,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,6,11,55.0,18.7,5.0,16.0,2,522,2,17,5.000000,70.000000,68.750000,5.000000,70.000000,68.750000,5.00,70.0,68.750000,5.000000,70.000000,68.750000,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,1,2,55.0,12.9,4.0,16.0,2,452,4,17,4.000000,110.000000,75.000000,4.000000,110.000000,75.000000,4.00,110.0,75.000000,4.000000,110.000000,75.000000,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,7,13,54.0,2.0,1.0,16.0,2,496,-8,17,2.250000,202.500000,76.515152,2.333333,176.666667,74.242424,2.25,202.5,76.515152,2.250000,202.500000,76.515152,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
202109050103,6,12,55.0,448.1,16.0,16.0,2,442,2,17,10.000000,0.000000,33.333333,10.000000,0.000000,33.333333,10.00,0.0,33.333333,10.000000,0.000000,33.333333,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202306010812,6,6,57.0,8.2,5.0,16.0,4,512,-4,9,6.000000,226.714286,53.350221,10.000000,0.000000,23.232323,7.00,266.0,46.356976,6.000000,226.714286,53.350221,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,7,7,56.0,4.3,2.0,16.0,5,476,12,9,5.083333,241.366667,64.518782,6.666667,202.000000,53.212670,5.40,275.2,57.344268,4.571429,285.200000,64.812810,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
202306010812,5,5,58.0,74.2,9.0,16.0,5,478,2,9,9.105263,110.105263,29.573774,9.000000,0.000000,36.698718,9.40,0.0,26.463675,10.571429,0.000000,21.680403,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,8,8,56.0,47.3,8.0,16.0,5,476,2,9,4.571429,206.700000,71.237245,8.666667,37.000000,42.592593,7.60,78.2,50.019841,6.000000,219.542857,61.125283,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [252]:
len(result_data)

57533

In [253]:
result_data

array([0.43207796, 0.20913978, 0.28305063, ..., 0.06539425, 0.10492492,
       0.09392313])

In [254]:
valid

,着順,枠番,馬番,斤量,単勝,人気,course_len,date,rank,age,weight,change_weight,n_horses,着順_allR,賞金_allR,着順/頭数_allR,着順_3R,賞金_3R,着順/頭数_3R,着順_5R,賞金_5R,着順/頭数_5R,着順_7R,賞金_7R,着順/頭数_7R,...,jockey_id_05622,jockey_id_05623,jockey_id_05624,jockey_id_05625,jockey_id_05626,jockey_id_05627,jockey_id_05628,jockey_id_05629,jockey_id_05630,jockey_id_05631,jockey_id_05632,jockey_id_05633,gender_セ,gender_牝,gender_牡,place_01,place_02,place_03,place_04,place_05,place_06,place_07,place_08,place_09,place_10
202109050103,1,2,4,55.0,5.7,3.0,16.0,2021-11-06,1,2,472,-4,17,7.500000,90.000000,47.500000,7.500000,90.000000,47.500000,7.50,90.0,47.500000,7.500000,90.000000,47.500000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,2,6,11,55.0,18.7,5.0,16.0,2021-11-06,1,2,522,2,17,5.000000,70.000000,68.750000,5.000000,70.000000,68.750000,5.00,70.0,68.750000,5.000000,70.000000,68.750000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,3,1,2,55.0,12.9,4.0,16.0,2021-11-06,1,2,452,4,17,4.000000,110.000000,75.000000,4.000000,110.000000,75.000000,4.00,110.0,75.000000,4.000000,110.000000,75.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
202109050103,4,7,13,54.0,2.0,1.0,16.0,2021-11-06,0,2,496,-8,17,2.250000,202.500000,76.515152,2.333333,176.666667,74.242424,2.25,202.5,76.515152,2.250000,202.500000,76.515152,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
202109050103,5,6,12,55.0,448.1,16.0,16.0,2021-11-06,0,2,442,2,17,10.000000,0.000000,33.333333,10.000000,0.000000,33.333333,10.00,0.0,33.333333,10.000000,0.000000,33.333333,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202306010812,5,6,6,57.0,8.2,5.0,16.0,2023-01-22,0,4,512,-4,9,6.000000,226.714286,53.350221,10.000000,0.000000,23.232323,7.00,266.0,46.356976,6.000000,226.714286,53.350221,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,6,7,7,56.0,4.3,2.0,16.0,2023-01-22,0,5,476,12,9,5.083333,241.366667,64.518782,6.666667,202.000000,53.212670,5.40,275.2,57.344268,4.571429,285.200000,64.812810,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
202306010812,7,5,5,58.0,74.2,9.0,16.0,2023-01-22,0,5,478,2,9,9.105263,110.105263,29.573774,9.000000,0.000000,36.698718,9.40,0.0,26.463675,10.571429,0.000000,21.680403,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
202306010812,8,8,8,56.0,47.3,8.0,16.0,2023-01-22,0,5,476,2,9,4.571429,206.700000,71.237245,8.666667,37.000000,42.592593,7.60,78.2,50.019841,6.000000,219.542857,61.125283,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [255]:
result_data

array([0.43207796, 0.20913978, 0.28305063, ..., 0.06539425, 0.10492492,
       0.09392313])

In [256]:
y_results_2 = [0 if p<0.5 else 1 for p in result_data]

In [257]:
pd.Series(y_results_2).value_counts()

0    50914
1     6619
dtype: int64

In [267]:
pred_table = valid[["着順","rank","人気"]]
pred_table

,着順,rank,人気
202109050103,1,1,3.0
202109050103,2,1,5.0
202109050103,3,1,4.0
202109050103,4,0,1.0
202109050103,5,0,16.0
...,...,...,...
202306010812,5,0,5.0
202306010812,6,0,2.0
202306010812,7,0,9.0
202306010812,8,0,8.0


In [268]:
pred_table["pred"] = y_results_2

C:\Users\81902\AppData\Local\Temp\ipykernel_22156\2182340785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_table["pred"] = y_results_2


In [269]:
pred_table

,着順,rank,人気,pred
202109050103,1,1,3.0,0
202109050103,2,1,5.0,0
202109050103,3,1,4.0,0
202109050103,4,0,1.0,1
202109050103,5,0,16.0,0
...,...,...,...,...
202306010812,5,0,5.0,0
202306010812,6,0,2.0,1
202306010812,7,0,9.0,0
202306010812,8,0,8.0,0


In [270]:
atari = pred_table[(pred_table["pred"] == 1) & (pred_table["rank"] == pred_table["pred"])]

In [271]:
kounyu =  pred_table[pred_table["pred"] == 1]

In [272]:
atari

,着順,rank,人気,pred
202109050106,1,1,1.0,1
202109050106,2,1,2.0,1
202109050107,1,1,2.0,1
202109050104,1,1,1.0,1
202109050105,1,1,1.0,1
...,...,...,...,...
202307010805,2,1,3.0,1
202307010804,2,1,3.0,1
202307010804,3,1,1.0,1
202306010812,1,1,3.0,1


In [273]:
atari["人気"].value_counts()

1.0    2508
2.0    1321
3.0     329
4.0      34
5.0       2
Name: 人気, dtype: int64

In [274]:
return_df = pd.read_pickle("../data/original_data/return_tables.pickle")

In [275]:
return_df

,0,1,2,3
202205050101,単勝,2,290,2
202205050101,複勝,2br8br3,150br810br210,2br7br3
202205050101,枠連,1 - 4,310,1
202205050101,馬連,2 - 8,9730,22
202205050101,ワイド,2 - 8br2 - 3br3 - 8,"2,010br420br3,560",19br4br28
...,...,...,...,...
202210040812,馬連,10 - 15,1150,2
202210040812,ワイド,10 - 15br3 - 15br3 - 10,"490br1,940br1,470",2br21br19
202210040812,馬単,15 → 10,2010,3
202210040812,三連複,3 - 10 - 15,7020,21


In [276]:
def tansho(df):
    tansho = df[df[0]=='単勝'][[1,2]]
    tansho.columns = ['win', 'return']
    
    for column in tansho.columns:
        tansho[column] = pd.to_numeric(tansho[column], errors='coerce')
        
    return tansho

In [277]:
p_return_df = tansho(return_df)

In [278]:
p_return_df

,win,return
202205050101,2.0,290.0
202205050102,10.0,770.0
202205050103,10.0,13120.0
202205050104,6.0,220.0
202205050105,16.0,1510.0
...,...,...
202210040808,8.0,280.0
202210040809,18.0,600.0
202210040810,6.0,1290.0
202210040811,1.0,740.0


In [279]:
tansho_kekka = atari[atari["着順"] == 1]

In [280]:
tansho_kekka

,着順,rank,人気,pred
202109050106,1,1,1.0,1
202109050107,1,1,2.0,1
202109050104,1,1,1.0,1
202109050105,1,1,1.0,1
202105050211,1,1,1.0,1
...,...,...,...,...
202306010803,1,1,1.0,1
202307010802,1,1,1.0,1
202307010801,1,1,1.0,1
202307010805,1,1,2.0,1


In [281]:
df_m = pd.merge(tansho_kekka, p_return_df, how="inner", left_index=True, right_index=True)

In [282]:
df_m

,着順,rank,人気,pred,win,return
202109050106,1,1,1.0,1,1.0,230.0
202109050107,1,1,2.0,1,5.0,420.0
202109050104,1,1,1.0,1,2.0,170.0
202109050105,1,1,1.0,1,5.0,250.0
202105050211,1,1,1.0,1,10.0,300.0
...,...,...,...,...,...,...
202306010803,1,1,1.0,1,4.0,190.0
202307010802,1,1,1.0,1,1.0,190.0
202307010801,1,1,1.0,1,9.0,120.0
202307010805,1,1,2.0,1,11.0,280.0


In [283]:
df_m["return"].sum()

547650.0

In [284]:
kounyu

,着順,rank,人気,pred
202109050103,4,0,1.0,1
202109050103,15,0,2.0,1
202105050112,4,0,1.0,1
202109050106,1,1,1.0,1
202109050106,2,1,2.0,1
...,...,...,...,...
202307010804,4,0,2.0,1
202306010811,5,0,1.0,1
202306010812,1,1,3.0,1
202306010812,3,1,1.0,1


In [286]:
kounyu[kounyu["人気"] != 1]

,着順,rank,人気,pred
202109050103,15,0,2.0,1
202109050106,2,1,2.0,1
202109050107,1,1,2.0,1
202109050104,4,0,2.0,1
202109050105,2,1,3.0,1
...,...,...,...,...
202307010805,2,1,3.0,1
202307010804,2,1,3.0,1
202307010804,4,0,2.0,1
202306010812,1,1,3.0,1


In [293]:
kounyu["人気"].value_counts()

1.0    3669
2.0    2265
3.0     634
4.0      49
5.0       2
Name: 人気, dtype: int64

In [ ]:
21400

In [294]:
tansho_without_1 = kounyu[(kounyu["人気"] != 1) & (kounyu["人気"] != 2) & (kounyu["rank"] == kounyu["pred"]) & (kounyu["着順"] == 1)]

In [295]:
df_m = pd.merge(tansho_without_1, p_return_df, how="inner", left_index=True, right_index=True)

In [296]:
df_m

,着順,rank,人気,pred,win,return
202105050205,1,1,3.0,1,1.0,480.0
202109050306,1,1,3.0,1,8.0,470.0
202105050410,1,1,3.0,1,7.0,440.0
202105050406,1,1,3.0,1,4.0,650.0
202105050506,1,1,3.0,1,9.0,390.0
...,...,...,...,...,...,...
202307010506,1,1,3.0,1,3.0,380.0
202307010701,1,1,3.0,1,13.0,430.0
202310010403,1,1,3.0,1,12.0,480.0
202310010407,1,1,3.0,1,4.0,420.0


In [299]:
kounyu[(kounyu["人気"] != 1) & (kounyu["人気"] != 2)]

,着順,rank,人気,pred
202109050105,2,1,3.0,1
202105050209,4,0,3.0,1
202103020201,8,0,3.0,1
202109050203,6,0,3.0,1
202105050205,1,1,3.0,1
...,...,...,...,...
202310010407,1,1,3.0,1
202306010807,5,0,3.0,1
202307010805,2,1,3.0,1
202307010804,2,1,3.0,1


In [297]:
df_m["return"].sum()

60530.0

In [300]:
68500 - 60530

7970

In [290]:
21400 - 17280

4120

In [291]:
247380 - 295000

-47620